In [3]:
from pyrosetta.rosetta.core.pack.task import *
from pyrosetta.rosetta.protocols import *
from pyrosetta.rosetta.core.select import *

#Python
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *

#Core Includes
from rosetta.core.kinematics import MoveMap
from rosetta.core.kinematics import FoldTree
from rosetta.core.pack.task import TaskFactory
from rosetta.core.pack.task import operation
from rosetta.core.simple_metrics import metrics
from rosetta.core.select import residue_selector as selections
from rosetta.core import select
from rosetta.core.select.movemap import *

#Protocol Includes
from rosetta.protocols import minimization_packing as pack_min
from rosetta.protocols import relax as rel
from rosetta.protocols.antibody.residue_selector import CDRResidueSelector
from rosetta.protocols.antibody import *
from rosetta.protocols.loops import *
from rosetta.protocols.relax import FastRelax
import multiprocessing
import pandas as pd
import numpy as np

from itertools import combinations 

from pyrosetta.toolbox import *

#init("-mute all")
init("-default_max_cycles 200 -ignore_unrecognized_res 1 -ex1 -ex2aro -ignore_zero_occupancy false -fa_max_dis 9")

PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python38.linux 2021.45+release.b2be42856e92b441d5d806df7b34ab4d77e39eed 2021-11-10T12:38:05] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python38.linux r302 2021.45+release.b2be428 b2be42856e92b441d5d806df7b34ab4d77e39eed http://www.pyrosetta.org 2021-11-10T12:38:05
core.init: command: PyRosetta -default_max_cycles 200 -ignore_unrecognized_res 1 -ex1 -ex2aro -ignore_zero_occupancy false -fa_max_dis 9 -database /home/david/anaconda3/envs/ros/lib/python3.8/site-packages/pyrosetta-2021.45+release.b2be428-py3.8-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-25928630 seed_offset=0 real_seed=-25928630
basic.random.init_random_generator: RandomGenerator:init: Norma

In [4]:
from pyrosetta.toolbox import cleanATOM
cleanATOM("ranked_0.pdb")

In [5]:
pose = pose_from_pdb("ranked_0.clean.pdb")

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 983 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.852502 seconds.
core.import_pose.import_pose: File 'ranked_0.clean.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 1 in file ranked_0.clean.pdb. Best match rsd_type:  MET:NtermProteinFull
core.conformation.Conformation: Found disulfide between residues 257 263
core.conformation.Conformation: current variant for 257 CYS
core.conformation.Conformation: current variant for 263 CYS
core.conformation.Conformation: current variant for 257 CYD
core.conformation.Conformation: current variant for 263 CYD


In [6]:
print(pose.pdb_info())

PDB file name: ranked_0.clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0318    A 0001  -- 0318  |   0318 residues;    04810 atoms
                           TOTAL |   0318 residues;    04810 atoms



In [7]:
scorefxn = create_score_function("ref2015_cart")

core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 9)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 9)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
basic.io.database: Database file opened: scoring/score_functions/rama/fd/all.ramaProb
basic.io.database: Database file opened: scor

In [8]:
scorefxn(pose)

core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (318)
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /home/david/anaconda3/envs/ros/lib/python3.8/site-packages/pyrosetta-2021.45+release.b2be428-py3.8-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/home/david/anaconda3/envs/ros/lib/python3.8/site-packages/pyrosetta-2021.45+release.b2be428-py3.8-linux-x86_64.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 libr

-600.4657698275661

In [9]:
ENZYME = "hg3"

num_iter = 20

lowest = -600
        
for i in range(num_iter):
    
    pose_relax = pose.clone()
    print("*"* 20,i,"*"* 20)
    scorefxn = create_score_function("ref2015_cart")
    relax = pyrosetta.rosetta.protocols.relax.FastRelax()

    relax.set_scorefxn(scorefxn)
        
    relax.cartesian(True)
        
    print(scorefxn(pose_relax), "BEFORE")

    relax.apply(pose_relax)

    print(scorefxn(pose_relax), "AFTER")
    
    pose_score = scorefxn(pose_relax)

    if pose_score < lowest:
        best_pose = pose_relax.clone()
        lowest = pose_score
        best_pose.dump_pdb(f"lowest_cart_relaxed_{ENZYME}.pdb")

******************** 0 ********************
protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /home/david/anaconda3/envs/ros/lib/python3.8/site-packages/pyrosetta-2021.45+release.b2be428-py3.8-linux-x86_64.egg/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: repack
protocols.r

In [10]:
print("Donw")

Donw
